In [1]:
pip install python-amazon-sp-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 4.6 MB/s eta 0:00:00


In [2]:
from sp_api.base import Marketplaces, ReportType
from sp_api.api import Orders
from sp_api.api import Reports

creds = dict(
    refresh_token='x',
    lwa_app_id='x',
    lwa_client_secret='x',
    aws_access_key='x',
    aws_secret_key='x'
) 



In [3]:
#inventory all listings report to get all ASINs under this account
from sp_api.api import Reports
import time

#code to get the key to download the report
report_types = "GET_MERCHANT_LISTINGS_ALL_DATA"
report = Reports(credentials=creds, marketplace=Marketplaces.US)
data = report.create_report(reportType=report_types)
report_id = data.payload['reportId'] 
print(data)
print('l\n')


#code to get the report
res = Reports(credentials=creds, marketplace=Marketplaces.US)
data = res.get_report(report_id)

while data.payload['processingStatus'] == "IN_QUEUE" or data.payload['processingStatus'] == 'IN_PROGRESS' :
  time.sleep(2)
  data = res.get_report(report_id)

if data.payload['processingStatus'] == "FATAL" or data.payload['processingStatus'] == "CANCELLED":
  print("Report failed!")
  report_data = data.payload
else:
  print("Success:")
  report_data = res.get_report_document(data.payload['reportDocumentId'])
  report_url = report_data.payload.get('url')
  print(report_data.payload)

{'errors': None,
 'headers': {'Server': 'Server', 'Date': 'Tue, 06 Jun 2023 22:02:29 GMT', 'Content-Type': 'application/json', 'Content-Length': '26', 'Connection': 'keep-alive', 'x-amz-rid': '3T5TGZ1RKHCHCR7P9V86', 'x-amzn-RateLimit-Limit': '0.0167', 'x-amzn-RequestId': 'ef3dd611-9238-481a-a187-c7b6efbff6ea', 'x-amz-apigw-id': 'OPFef3dd6119238', 'X-Amzn-Trace-Id': 'Root=1-647facf5-ef3dd6119238481a', 'Vary': 'Content-Type,Accept-Encoding,User-Agent', 'Strict-Transport-Security': 'max-age=47474747; includeSubDomains; preload'},
 'next_token': None,
 'pagination': None,
 'payload': {'reportId': '70692019514'},
 'rate_limit': '0.0167'}
l

Success:
{'reportDocumentId': 'amzn1.spdoc.1.4.na.b2fa9294-c154-448f-bdcf-e6f8214942a5.TX7CWY2AE3ZUJ.47700', 'url': 'https://tortuga-prod-na.s3-external-1.amazonaws.com/%2FNinetyDays/amzn1.tortuga.4.na.eea9aa4c-96ea-4c44-b377-a93d7cab0620.T2TR5IZOS45I4S?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Date=20230606T220247Z&X-Amz-SignedHeaders=host&X-Amz-Expires=30

In [4]:
import requests
import csv


#code to make the data in a more disgesale manner/dictionary
url_request = requests.get(report_url)
decoded_content = url_request.content.decode('cp1252') #string type
reader = csv.DictReader(decoded_content.splitlines(), delimiter='\t') #csv.DictReader type


data_list = []
for row in reader:
  print(row)
  data = {
      'item-name': row['item-name'],
      'item_description': row['item-description'],
      'listing-id': row['listing-id'],
      'seller-sku': row['seller-sku'],
      'open-date': row['open-date'],
      'product-id': row['product-id'] 
      }
  data_list.append(data)

{'item-name': 'Optimized Athletics Psoas Release Tool I Stretching Tool and Deep Tissue Relief I Hip Hook & Hip Flexor Release I Mitigate Soreness of Hip Flexor & Psoas Muscle I Red', 'item-description': 'Stretcher', 'listing-id': '1201ZD62SSY', 'seller-sku': 'Q9-649U-11YV', 'price': '36.99', 'quantity': '', 'open-date': '2022-12-01 11:48:54 PST', 'image-url': '', 'item-is-marketplace': 'y', 'product-id-type': '1', 'zshop-shipping-fee': '', 'item-note': '', 'item-condition': '11', 'zshop-category1': '', 'zshop-browse-path': '', 'zshop-storefront-feature': '', 'asin1': 'B0BNR6J3BB', 'asin2': '', 'asin3': '', 'will-ship-internationally': '', 'expedited-shipping': '', 'zshop-boldface': '', 'product-id': 'B0BNR6J3BB', 'bid-for-featured-placement': '', 'add-delete': '', 'pending-quantity': '', 'fulfillment-channel': 'AMAZON_NA', 'merchant-shipping-group': 'Migrated Template', 'status': 'Active'}
{'item-name': 'Psoas Release Tool I Optimized Athletics -Stretching Tool and Deep Tissue Relief 

In [6]:
from datetime import datetime

# Starting datetime
starting_datetime = datetime(2022,10,1)
starting_datetime_iso = starting_datetime.strftime('%Y-%m-%dT%H:%M:%S-05:00')
print('Starting Datetime:', starting_datetime)
print('Starting Datetime ISO: ', starting_datetime_iso)
print("\n")


# Ending datetime
ending_datetime = datetime(2023,7,1)
ending_datetime_iso = ending_datetime.strftime('%Y-%m-%dT%H:%M:%S-05:00')
print('Ending Datetime:', ending_datetime)
print('Ending Datetime ISO:', ending_datetime_iso)

Starting Datetime: 2022-10-01 00:00:00
Starting Datetime ISO:  2022-10-01T00:00:00-05:00


Ending Datetime: 2023-07-01 00:00:00
Ending Datetime ISO: 2023-07-01T00:00:00-05:00


In [7]:
#the below code breaks down month by asin 

from sp_api.api.sales.sales import Granularity
from sp_api.api import Sales
from sp_api.base import Granularity
import pandas as pd


df0 = pd.DataFrame(columns=['ASIN',"Item Name", "Date", "Ordered Units", "Ordered Revenue"])


count = 0
for i in data_list:
  product_id = i['product-id']
  item_name = i['item-name']
  sales = Sales(credentials=creds, marketplace=Marketplaces.US)
  sales_request = sales.get_order_metrics(marketplaceIds="ATVPDKIKX0DER", interval=(f"{starting_datetime_iso}",f"{ending_datetime_iso}"),granularity=Granularity.MONTH, asin=product_id)
 
  z = 0
  for x in sales_request.payload:
    #print(product_id)
    #print(x['interval'])
    #print(sales_request.payload[z]['unitCount'])
    #print(sales_request.payload[z]['totalSales']["amount"])


    #first covnerting a date like 2022-10-01T00:00:00-05:00 into October 2022
    iso_date = sales_request.payload[z]['interval'].split('--')[0]
    iso_date_to_datetime =datetime.strptime(iso_date,'%Y-%m-%dT%H:%M-05:00')
    human_readable_date = iso_date_to_datetime.strftime('%B %Y') 

    #add data to the tabl
    data = {'ASIN': product_id,
            'Item Name': item_name,
            'Date':human_readable_date, 
            'Ordered Units':(sales_request.payload[z]['unitCount']), 
            'Ordered Revenue': (sales_request.payload[z]['totalSales']["amount"])}
    df0 = df0.append(data, ignore_index = True)

    z = z + 1
  count = count + 1

#printing how it looks
display(df0)

#code to reformat it the way customers want it
df0_reformatted = df0.pivot(index=['ASIN', "Item Name"], columns='Date')['Ordered Revenue']
display(df0_reformatted)

  


<ipython-input-7-757ec55f5aa4>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df0 = df0.append(data, ignore_index = True)
<ipython-input-7-757ec55f5aa4>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df0 = df0.append(data, ignore_index = True)
<ipython-input-7-757ec55f5aa4>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df0 = df0.append(data, ignore_index = True)
<ipython-input-7-757ec55f5aa4>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df0 = df0.append(data, ignore_index = True)
<ipython-input-7-757ec55f5aa4>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

,ASIN,Item Name,Date,Ordered Units,Ordered Revenue
0,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,October 2022,0,0.00
1,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,November 2022,0,0.00
2,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,December 2022,0,0.00
3,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,January 2023,155,5437.46
4,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,February 2023,77,3329.23
5,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,March 2023,215,7485.86
6,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,April 2023,143,4588.57
7,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,May 2023,194,6932.06
8,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,June 2023,48,1677.52
9,B0B8TGRN5L,Psoas Release Tool I Optimized Athletics -Stre...,October 2022,1002,41733.99


,Date,April 2023,December 2022,February 2023,January 2023,June 2023,March 2023,May 2023,November 2022,October 2022
ASIN,Item Name,,,,,,,,,
B0B8TGRN5L,Psoas Release Tool I Optimized Athletics -Stretching Tool and Deep Tissue Relief I Hip Hook & Hip Flexor Release I Mitigate Muscle Soreness of Hip Flexor & Psoas Muscle,50004.29,49311.7,55986.11,61130.74,6213.45,47588.10,52346.91,43548.09,41733.99
B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stretching Tool and Deep Tissue Relief I Hip Hook & Hip Flexor Release I Mitigate Soreness of Hip Flexor & Psoas Muscle I Red,4588.57,0.0,3329.23,5437.46,1677.52,7485.86,6932.06,0.00,0.00


In [8]:
from sp_api.api import Reports
import time

#code to get the key to download the rreport
report_types = "GET_SALES_AND_TRAFFIC_REPORT"
report = Reports(credentials=creds, marketplace=Marketplaces.US)
data = report.create_report(reportType=report_types, dataStartTime='2022-10-01T00:00:00.000Z', dataEndTime ="2022-12-31T00:00:00.000Z", reportOptions={"dateGranularity":"MONTH"})
report_id = data.payload['reportId'] #this doesn't work for some reason
print(data)
print(report_id)
print('l\n')


#code to get the report
res = Reports(credentials=creds, marketplace=Marketplaces.US)
data = res.get_report(report_id)

while data.payload['processingStatus'] == "IN_QUEUE" or data.payload['processingStatus'] == 'IN_PROGRESS' :
  time.sleep(2)
  data = res.get_report(report_id)

if data.payload['processingStatus'] == "FATAL" or data.payload['processingStatus'] == "CANCELLED":
  print("Report failed!")
  report_data = data.payload
else:
  print("Success:")
  report_data = res.get_report_document(data.payload['reportDocumentId'])
  report_url = report_data.payload.get('url')
  print(report_data.payload)


{'errors': None,
 'headers': {'Server': 'Server', 'Date': 'Tue, 06 Jun 2023 22:05:32 GMT', 'Content-Type': 'application/json', 'Content-Length': '26', 'Connection': 'keep-alive', 'x-amz-rid': '454PSQ3WD16CVGFNQ5NY', 'x-amzn-RateLimit-Limit': '0.0167', 'x-amzn-RequestId': '58f20313-1cd1-453a-8ec9-04f699b4b046', 'x-amz-apigw-id': 'OPF58f203131cd1', 'X-Amzn-Trace-Id': 'Root=1-647fadac-58f203131cd1453a', 'Vary': 'Content-Type,Accept-Encoding,User-Agent', 'Strict-Transport-Security': 'max-age=47474747; includeSubDomains; preload'},
 'next_token': None,
 'pagination': None,
 'payload': {'reportId': '70693019514'},
 'rate_limit': '0.0167'}
70693019514
l

Success:
{'reportDocumentId': 'amzn1.spdoc.1.4.na.2d924199-8c2b-4e23-b8e8-56d0eb9d1bdf.T1P54NZWS5Q9RP.44900', 'compressionAlgorithm': 'GZIP', 'url': 'https://tortuga-prod-na.s3-external-1.amazonaws.com/%2FNinetyDays/amzn1.tortuga.4.na.c848834e-7aa7-4651-935d-86fb93f841b2.T22MGST7KHEAUU?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Date=20230606T2205

In [9]:
from re import I
import requests
import csv
import json
from urllib.request import urlopen
import gzip



#code to make the data in a more disgesale manner/dictionary
url_request = requests.get(report_url)
decoded_content = gzip.decompress(url_request.content)
content = json.loads(decoded_content)


df2 = pd.DataFrame(columns=['Date', "PageViews", "Sessions", "Conversion Rate"])


x = 0
for i in content['salesAndTrafficByDate']:


  #first covnerting a date like 2022-10-01 into October 2022
  date = content['salesAndTrafficByDate'][x]['date']
  iso_date_to_datetime =datetime.strptime(date,'%Y-%m-%d')
  human_readable_date = iso_date_to_datetime.strftime('%B %Y') 

  data = {'Date':human_readable_date,
          'PageViews': (content['salesAndTrafficByDate'][x]['trafficByDate']['pageViews']),
          'Sessions': content['salesAndTrafficByDate'][x]['trafficByDate']['sessions'],
          'Conversion Rate': content['salesAndTrafficByDate'][x]['trafficByDate']['unitSessionPercentage']
          }
  df2 = df2.append(data, ignore_index = True)


  x = x + 1


<ipython-input-9-ef52f66f9e9c>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(data, ignore_index = True)
<ipython-input-9-ef52f66f9e9c>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(data, ignore_index = True)
<ipython-input-9-ef52f66f9e9c>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(data, ignore_index = True)


In [10]:
print(df0.dtypes) # somehow Ordered Units is an object
display(df0)


df0['Ordered Units'] = df0['Ordered Units'].astype(float)
df0 = df0.groupby("Date").mean()
display(df0)

ASIN                object
Item Name           object
Date                object
Ordered Units       object
Ordered Revenue    float64
dtype: object


,ASIN,Item Name,Date,Ordered Units,Ordered Revenue
0,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,October 2022,0,0.00
1,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,November 2022,0,0.00
2,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,December 2022,0,0.00
3,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,January 2023,155,5437.46
4,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,February 2023,77,3329.23
5,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,March 2023,215,7485.86
6,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,April 2023,143,4588.57
7,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,May 2023,194,6932.06
8,B0BNR6J3BB,Optimized Athletics Psoas Release Tool I Stret...,June 2023,48,1677.52
9,B0B8TGRN5L,Psoas Release Tool I Optimized Athletics -Stre...,October 2022,1002,41733.99


<ipython-input-10-052650a4e3a8>:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df0 = df0.groupby("Date").mean()


,Ordered Units,Ordered Revenue
Date,,
April 2023,707.5,27296.430
December 2022,615.5,24655.850
February 2023,734.5,29657.670
January 2023,841.5,33284.100
June 2023,101.5,3945.485
March 2023,704.0,27536.980
May 2023,753.5,29639.485
November 2022,546.5,21774.045
October 2022,501.0,20866.995


In [11]:
import pandas as pd
from IPython.display import display
pd.set_option('float_format', '{:,.1f}'.format)


#merging the two datasets
df3 = pd.merge(df0,df2,on='Date')
#df3.groupby('Date', axis=0)
#df3.groupby(pd.Grouper(key='Date', axis=0, freq='M'))



#adding MoM metrrics

df3['Ordered Revenue MoM'] = (df3['Ordered Revenue'].pct_change()*1)
df3['Ordered Revenue MoM'] = ["{:,.1%}".format(i) for i in df3['Ordered Revenue MoM']]


df3['Ordered Units MoM'] = (df3['Ordered Units'].pct_change()*1)
df3['Ordered Units MoM'] = ["{:,.1%}".format(i) for i in df3['Ordered Units MoM']]


df3['Sessions MoM'] = (df3['Sessions'].pct_change()*1)
df3['Sessions MoM'] = ["{:,.1%}".format(i) for i in df3['Sessions MoM']]



display(df0)
display(df2)
display(df3)


,Ordered Units,Ordered Revenue
Date,,
April 2023,707.5,"27,296.4"
December 2022,615.5,"24,655.8"
February 2023,734.5,"29,657.7"
January 2023,841.5,"33,284.1"
June 2023,101.5,"3,945.5"
March 2023,704.0,"27,537.0"
May 2023,753.5,"29,639.5"
November 2022,546.5,"21,774.0"
October 2022,501.0,"20,867.0"


,Date,PageViews,Sessions,Conversion Rate
0,October 2022,9003,6407,15.6
1,November 2022,11134,7817,14.0
2,December 2022,11289,7753,15.9


,Date,Ordered Units,Ordered Revenue,PageViews,Sessions,Conversion Rate,Ordered Revenue MoM,Ordered Units MoM,Sessions MoM
0,December 2022,615.5,"24,655.8",11289,7753,15.9,nan%,nan%,nan%
1,November 2022,546.5,"21,774.0",11134,7817,14.0,-11.7%,-11.2%,0.8%
2,October 2022,501.0,"20,867.0",9003,6407,15.6,-4.2%,-8.3%,-18.0%


In [ ]:
#export our work
from google.colab import files

df0_reformatted.to_csv('Monthly_Revenue_By_ASIN.csv', encoding='utf-8')
files.download('Monthly_Revenue_By_ASIN.csv')

df3.to_csv(r'Monthly_KPIs_Performance.csv', encoding='utf-8')
files.download('Monthly_KPIs_Performance.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>